In [2]:
import zlib
import codecs
cTypes = ['GZip','Zlib']

In [3]:
#prep region file
worldName = "TestZone"
regionFileName = "r.0.0.mca"
chunk  = [0,0] #[x,z]
yindex =  0

regionFilePath = "C:\\Users\\craz0\\AppData\\Roaming\\.minecraft\\saves\\"+worldName+"\\region\\"+regionFileName

In [4]:
#read region file for chunch data
offset = 4 * ((chunk[0] % 32) + (chunk[1] % 32) * 32)
print("Location Offset: " + str(offset))
with open(regionFilePath,"rb") as file:
    file.seek(offset,0)
    chunkOffset = 4096*int(file.read(3).hex(),16)
    chunkSize = 4096*int(file.read(1).hex(),16)
    print("Chunck Offset: " + str(chunkOffset))
    print("Chunck Size: " + str(chunkSize))
    file.seek(chunkOffset,0)
    chunkLength = int(file.read(4).hex(),16)
    print("Chunck Length: " + str(chunkLength))
    cType = int(file.read(1).hex(),16)
    print("Compression Type: " + cTypes[cType-1])
    cChunkData = file.read(int(str(chunkLength),16)-1).hex()
    print(cChunkData)

Location Offset: 0
Chunck Offset: 8192
Chunck Size: 4096
Chunck Length: 964
Compression Type: Zlib
789cedd9414e1b5700c7e1e740000f8624cbee7b8aee809804d5c5083ba92a554213fb05468087ce0c51d30bf42e39554e50f506edbc40a8145542d5d0b0e0fb495ef062f81c273263ffb310b2f07814dfc5b3b5b03269f2e6b20e1b1765dd5c54e52cd6759c2f85e5df0eca3ab42d876c94d7cdab8b79dec4f0a90fbfaf8795eda23c8fed3d7ae9a4e7e6e6f6306fcb61636f7192bf299a389f16e7d72f12fd45fb22f2ebf58b4816b297b1383e69cef38b7a10d6c73bc3adfda3ddd1787cd8fee9b757dfd10bfaf706e19b1fc6d3bdf1fed1f668bcf3fddefe8ba3fdf1d168b8f57a38099ebf5b1b84275f3c7fc1b3766b83b0f1e3f870f4fc68f2ea70776b67183c67b736089ba3bd172fa7fea3fd97fa61302dcee270d1144d11ebebb3b52fbecea6e5769c16b3d338efb7074feffe51f4d3a348c2cd4398c45953948b3a4b60fb8b6bfbac9c9da64be69b4bdfd47dffd3f2f97c3e9fcfff8af5c3ea417e169b26a60b84476b61793f6fdf006f9c178b38abf2b7cd77795185cfc7cffe397e13e7557b251156db4b8cd3f7a3f4d638b4f7fbf8c7fdf6e75ff71b9fcfe7f3f90fd1ef85de4fed3541f6e963869bab024992f400333cf0f97c3e9fcfbffbe1e1be3

In [5]:
#unzip
BAchunkData = bytearray.fromhex(cChunkData)
BOchunkData = zlib.decompress(BAchunkData)
chunkData = BOchunkData.hex()

In [6]:
file = open("C:\\Users\\craz0\\Desktop\\chunkBase.hex","w")
file.write(chunkData)
file.close()

In [9]:
#frontAddress for block states
blockstateTag = "0c000b426c6f636b537461746573"
frontAdd = chunkData.index(blockstateTag)+len(blockstateTag)
lengthBlockStates = chunkData[frontAdd:frontAdd+8]
frontAdd += 8
endAdd = frontAdd+(int(lengthBlockStates,16)*16)
chunkData[frontAdd:endAdd] #blockStates

'000000000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [31]:
#create new block states
blockStates = "0"*4096

blockStates

'000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [32]:
#replace blockStates
chunkData = chunkData.replace(chunkData[frontAdd:endAdd],blockStates)
    

In [33]:
#compress
chunkField = ""
BOchunkData = bytearray.fromhex(chunkData)
cChunkData = zlib.compress(BOchunkData)

chunkLength = hex(int(len(cChunkData.hex())/2))
if len(chunkLength)%2==0:
    chunkLength = "0"*(8-len(chunkLength[2:]))
else:
    chunkLength = "0"+str(chunkLength[2:])
    chunkLength = "0"*(8-len(chunkLength))+chunkLength
chunkLength = bytes.fromhex(chunkLength)


In [34]:
#overwrite chunk data [0,0]
with open(regionFilePath,"r+b") as file:
    file.seek(offset,0)
    chunkOffset = 4096*int(file.read(3).hex(),16)
    chunkSize = 4096*int(file.read(1).hex(),16)
    print("Chunck Offset: " + str(chunkOffset))
    print("Chunck Size: " + str(chunkSize))
    file.seek(chunkOffset,0)
    file.write(chunkLength)
    file.seek(1,1)
    file.write(cChunkData)
    file.seek(chunkOffset,0)
    print(file.read(int(str(chunkLength.hex()),16)+1))

Chunck Offset: 8192
Chunck Size: 4096
b'\x00\x00\x01\xfd\x02x\x9c\xed\x9aQo\xda0\x10\xc7\x0f\x9aAbBG\xa5\xbd\xec\xad\x0f\xfb\x14{*\xd0\xd0\xa2\xa5\t"i\xa7I\x93"7\xb9\x82E\x88il\xa6\xb5O\xfd*\xfb\xa4\x9b\x03i7\xf5\x89I\xeb:\xad\xf7\x93N\x7f\xf9\xe2\xd8\xd6)\xb2\xcf\xb1\x19\x00\x83W>~\xc1\xdc\x86V\xa4\xb9^+\xe8\xae\xa4\xd2\xabR\xa6\xa8\x14f{`\xddN\xa4\x02\x83\x05\xcc\xe7J\x9f\xaf2\xae\x116|{\xd3\x81\xd6@\xc8%\x9a\x1a\x8d\xca\xd3 ##{\x99fAw\\\xcc\xf9\xa5\xd0\x98\xc5bYO\x12\x00f\x12\xf9ZO"\x0c\xd8)\x8a\xd9\\/\xf9J\xb9\xd0\t\x87^?HF~\x18N\xcd\xd3wwG\x87=\xdbj6j\x81Z\xeej9\xaa\xe5\xb0\x96^-v-V-\xcd\xff\xb5\x1d\x17\xde\x9e\x85\xf18\x0c\x92\x81\x1f\x0e?\x8c\x83\x93$\x08\x13\xdf\xeb_x\x11\xc5o\x97\xf8\xbd~\x14?\x8a\xda.Q\xeb~\x0c\xa7\xfeq\x12\x9dOG\xfd\xa1G1\xdb%f\xfb\xfe\xf8\xe44\xa6\x0f\xed7\xda\x01\x07\xdcX\xe4\xe8\x15Zh\x81\x9b\x15\xc3\xf8\xecGe\x16\xcb\x01\xc6"]`\xe6\x18G\x0f\xfe0\x0e8\xd5(\xaa\x1e\x1e\x86\x10a\xaa\x85,\x14\xab\x16:\xb3p\rr\x99.\xaa\x94\xf9!\xf5%\x08\x82 \x08\xe2\x85\xe1@{